### 1. Deep Q-learning

The idea behind deep Q-learning (DQL) is to approximate the optimal action-value function $q_{*}$ with a deep neural network (DNN)  known as the __Q-network__ (as opposed to using a Q-table which keeps track of the maximum expected value from each action in each possible state). 

Apart from the main idea of using a DNN to to approximate $q_{*}$, deep Q-learning has two additional important features which allow for stable convergence to $q_{*}$:

- Experience replay 
- Fixed targets

__Eperience replay__ boils down to keeping a replay buffer of state, action, reward and next-state tuples $(S,A,R,S')$ which are gradually added to the buffer as the agent explores the environment. Since the agent sequentially interacts with the environment the tuples in the replay buffer can be highly correlated. The idea of __experience replay__ is to randomly sample batches of tuples from the replay buffer which breaks harmful correlations. Additionally, randomly sampling from the replay buffer allows the agent to learn from individual tuples multiple times potentially allowing the agent to recall rare occurences and overall make better use of experience. 

__Fixed Q-targets__ boils down to using a separate DNN whose weights are kept fixed over pre-defined stretches of learning steps in the algorithm. This network is known as the __target Q-network__ and has an identical architecture to the __Q-network__ ($\hat{q}$) but its weights $w^{-}$ are updated slower than the weights $w$ of the __Q-network__ because they are used as fixed targets in the update step for $w$

$$ \Delta w = \alpha \left( \underbrace{R + \gamma \max_{a} \hat{q}(S',a,w^{-})}_{Target} - \hat{q}(S',a,w^{-}) \right) \nabla_{w} \hat{q}(S,A,w) $$

This prevents the target function from changing too quickly.

In pseudo code the deep-Q learning algorithm can be summarized as follows:

Initialize network $\hat{q}$ with weights $w$.

Initialize network $\hat{tq}$ with weights $w^{-} \leftarrow w$. 

Initalize replay buffer with size $D$.

&emsp; __for__ episode $e \leftarrow$ 1 to $M$:

&emsp; &emsp; Prepare initial state $S$

&emsp; &emsp; __for__ time step $t \leftarrow $ 1 to $M$:

&emsp; &emsp; &emsp; __Sample phase__

&emsp; &emsp; &emsp; Choose action $A$ from state $S$ using $\pi \leftarrow \epsilon$-Greedy($\hat{q}(S,A,w)$)

&emsp; &emsp; &emsp; Take action $A$, observe reward $R$ and next state $S'$

&emsp; &emsp; &emsp; Store experince tuple $(S,A,R,S')$ into memory $D$

&emsp; &emsp; &emsp; $S \leftarrow S'$

&emsp; &emsp; &emsp; __Learning phase__

&emsp; &emsp; &emsp; Obtain random minibatch of tuples $\left\{(s_j, a_j, r_j, s_{j+1})\right\}_{j=1}^{J}$ from $D$

&emsp; &emsp; &emsp; Set target $y_j = r_j + \gamma \max_{a}\hat{q}(s_{j+1},a,w^{-})$

&emsp; &emsp; &emsp; Update $\Delta w = \alpha \left(y_j - \hat{q}(S',a,w^{-}) \right) \nabla_{w} \hat{q}(S,A,w)$

&emsp; &emsp; &emsp; Every $C$ steps update $w^{-} \leftarrow {w}$


#### 1.1 Hyperparameters

The state and action vector sizes for the Banana virtual environment are 37 and 4 respectively. The Q-network and the target Q-networks are both DNNs of size 37-128-64-4. The Adam optimization algorithm was used for optimizing the network weights with respect to the mean square loss function (```loss = F.mse_loss(Q_expected, Q_targets)```). All remaining hyper parameters are listed below:

```
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the target network  

n_episodes=2000         # max number of training episodes
max_t=1000              # max time steps per episode
eps_start=1.0           # eps_start (float): starting value of epsilon, for epsilon-greedy action selection
eps_end=0.01            # minimum value of epsilon
eps_decay=0.995         # multiplicative factor (per episode) for decreasing epsilon
```


### 2. Results on the Banana virtual enviroment

The learning rate of the DQN agent in the Banana environment is plotted below:

<img src='DQN_training.png'>

### 3. Future work 

While the agent learns to solve the task quite quickly - an average score of 13.0 is achieved in around 520 to 530 episodes the graph of the scores for each episode shows that learning is somewhat unstable. The agent's performance on the task can be further improved by using any of the following algorithms which extend the ideas in DQN: 

- [Prioritized experience replay](https://arxiv.org/abs/1511.05952) 
- [Dueling DQNs](https://arxiv.org/abs/1511.06581)
- [Double DQN](https://arxiv.org/abs/1509.06461)
